# コンテンツ
1. LoRAとはなにか
2. LoRAの主要なハイパーパラメータ
3. LoRAはなぜ有効なのか
4. おまけ：QLoRAとは

# LoRAとはなにか
Low Rank Adaptationの略(Rankは線形代数でよく出てくる行列の階数的なアレ)
ディープラーニングモデルで学習可能なパラメータ数を限定して計算量を削減するテクニック

## 通常のパラメータ学習
![](../images/visual_gpt2.png)
- たとえば上図はGPT系モデルの概要である
- そのうちの一つの層、Feed Forward層は線形変換

![](../images/dn.png)
- FeedForward層(リニア層)の場合、パラメータ数はシンプルに`入力ノード数×出力ノード数`になる。上記の例では最初のリニア層のパラメータ数は12個。

- LLMの入力は１つのトークンを埋め込みベクトルとして表現したもの、出力は同じ次元数のベクトル
- つまり埋め込みベクトルが1600次元なら、パラメータは(1600,1600)行列Wで表現できる
- 学習時はこのパラメータがすべて(256,000個！)少しずつ変化していく

## LoRA導入時のパラメータ学習
- この(1600,1600)行列`W`をと(16,1600)行列`A`と(1600,16)行列`B`の積で表したら、学習パラメータ数が51,200個(1600×16×2)まで削減できますやん、という発想。

![](../images/lora.PNG)

この図だと`d=1600`、`r=16`
- 左側の青い1層のパラメータは学習させない。固定する。
- 右側の赤い2層(これをAdapterという)をかっぽりとはめこみ、学習を担当させる。
$h = (W + \Delta W) x$
- 行列`A`、`B`ともに高々ランク`r`なのでもとの$\delta W$も高々ランク`r`($r << d$)
- これがLow rank adaptationの由来(低ランク行列をはめ込む)


# LoRAの主要なハイパーパラメータ
## 1.$ｒ$
- 最重要ハイパーパラメータ
- ただどうやって決めればいいのかわからない。誰も教えてくれない。
- peftライブラリだとデフォルト値が`8`になっており、そんな小さくして大丈夫なのかと勝手に思っている

## 2. $\alpha$
- 通常の学習における学習率$\eta$に相当するハイパーパラメータ
- 学習率とは、勾配$\delta$(修正する方向)を求めた後に、どれだけ進むかを表す指標
- LoRAにおいては通常の学習率を適用して$\delta W$行列を更新した後に、さらにそれをスケーリングするようになっている(たぶん)。実際には$\delta W$は$\frac{\alpha}{r}$倍される(つまりrが大きいときはスケーリングファクターはそのぶん小さくなるよう調整される)
- peftではデフォルト値は8なので、スケーリングファクターは1になる。つまりスケールしない。

# なぜLoRAは有効なテクニックなのか
- LoRAでは実際に$W$の勾配行列$\Delta W$を計算する代わりに
それよりも低ランクの$\Delta W'$で近似している。この近似に妥当性があるときにかぎりLoRAは有効であるといえる

- 結論としてはLLMのリニア層では、ひとつだけをとってみれば1600次元もの変換は起こっておらず、それよりもはるかに小さい次元での変換のみが起こっていることが多いらしい。そのためLoRAは有効であるといえる

- 以下では低ランク行列近似を実際のコードで示す。ポイントは行列の特異値分解である

In [2]:
from math import sqrt

import numpy as np

# 行列の階数チェック
def check_the_rank_of_BA():
    A = np.random.rand(5, 20)
    B = np.random.rand(20, 5)
    W_delta = B @ A
    print("行列Aの階数:", np.linalg.matrix_rank(W_delta))
    print("行列Aの階数:", np.linalg.matrix_rank(W_delta))
    print("行列dWの階数:", np.linalg.matrix_rank(W_delta))

check_the_rank_of_BA()

行列Aの階数: 5
行列Aの階数: 5
行列dWの階数: 5


In [6]:
# 特異値分解
# すべての行列はU(直行行列) * S(対角行列) * V(直行行列)に分解できる
def svd_test():
    w = np.random.rand(20, 20)
    u, s, vh = np.linalg.svd(w)
    print(u.shape)
    print(s.shape)
    print(vh.shape)

svd_test()

(20, 20)
(15,)
(15, 15)


In [ ]:
# 行列の低ランク行列近似
def low_rank_matrix_approx(n, rank):
    # 行列の生成
    w_original = np.random.rand(n, n)
    # 特異値分解
    u, s, vh = np.linalg.svd(w_original)
    s = np.diag(s)
    # 低ランク行列近似
    ur = u[:, :rank]
    sr = s[:rank, :rank]
    vhr = vh[:rank, :]
    w_low_rank = ur @ sr @ vhr
    # 誤差の計算
    error = np.linalg.norm(w - w_low_rank)
    print("w: ", w_original)
    print("w_low_rank: ", w_low_rank)
    print("誤差:", error)

low_rank_matrix_approx(5, 4)

In [ ]:
from PIL import Image

def adapt_low_rank_matrix_to_image(path):
    im = Image.open(path).convert('L')
    pix = np.array(im)
    print(pix.shape)
    u, s, vh = np.linalg.svd(pix)
    ranks = [1, 5, 10, 20, 50, 100, 200, 300]
    for rank in ranks:
        ur = u[:, :rank]
        sr = np.diag(s[:rank])
        vhr = vh[:rank, :]
        pix_after = ur @ sr @ vhr
        res = Image.fromarray(pix_after)
        print("rank:", rank)
        res.show()

adapt_low_rank_matrix_to_image("../images/example_for_Lora.png")

# 参考文献・URL
- [LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS](https://arxiv.org/pdf/2106.09685.pdf)
- [Understanding LoRA and QLoRA — The Powerhouses of Efficient Finetuning in Large Language Models](https://medium.com/@gitlostmurali/understanding-lora-and-qlora-the-powerhouses-of-efficient-finetuning-in-large-language-models-7ac1adf6c0cf)